In [ ]:
!pip install tfx==0.22.0

In [ ]:
import os
import sys

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen

In [ ]:
context = InteractiveContext(pipeline_root='../tfx')

In [ ]:
base_dir = os.getcwd()
data_dir = "../data"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 64
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/CsvExampleGen/examples/64, id: 78)]
        ))

In [ ]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 65
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ../tfx/StatisticsGen/statistics/65, id: 79)]
        ))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 66
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ../tfx/SchemaGen/schema/66, id: 80)]
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,optional,single,'state'
'sub_issue',STRING,optional,single,'sub_issue'
'sub_product',STRING,optional,single,'sub_product'
'timely_response',STRING,required,,'timely_response'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'state',"'AA', 'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'sub_issue',"'Account status', 'Account terms', 'Account terms and changes', 'Applied for loan/did not receive money', 'Attempted to collect wrong amount', 'Attempted to/Collected exempt funds', 'Billing dispute', 'Called after sent written cease of comm', 'Called outside of 8am-9pm', 'Can\'t contact lender', 'Can\'t decrease my monthly payments', 'Can\'t get flexible payment options', 'Can\'t qualify for a loan', 'Can\'t stop charges to bank account', 'Can\'t temporarily postpone payments', 'Charged bank acct wrong day or amt', 'Charged fees o

In [ ]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'issue',Unexpected string values,Examples contain values missing from the schema: Lender damaged or destroyed vehicle (<1%).


In [ ]:
transform_file = os.path.join(base_dir, '../components/module.py')

In [ ]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

feature {
  name: "state"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "state"
  presence {
    min_count: 1
  }
}
feature {
  name: "sub_issue"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_issue"
  presence {
    min_count: 1
  }
}
feature {
  name: "sub_product"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_product"
  presence {
    min_count: 1
  }
}
feature {
  name: "timely_response"
  type: BYTES
  domain: "timely_response"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "zip_code"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_count: 1
  }
}
feature {
  name: "consumer_disputed"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "company_response"
  value: "Closed"
  value: "Closed with explanation"
  v

ExecutionResult(
    component_id: Transform
    execution_id: 68
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ../tfx/Transform/transform_graph/68, id: 82)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/Transform/transformed_examples/68, id: 83)]
        ))

In [ ]:
trainer_file = os.path.join(base_dir, '../components/module.py')

In [ ]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
consumer_complaint_narrative_xf [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None,)]            0           consumer_complaint_narrative_xf[0
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 512)          256797824   tf_op_layer_Reshape[0][0]        
__________________________________________________________________________________________________
reshape (Reshape)               (None, 512)          0           keras_layer[0][0]                
______________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 70
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ../tfx/Trainer/model/70, id: 85)]
        ))

### Load TensorBoard

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

In [ ]:
log_dir = os.path.join(model_artifact_dir, 'logs/')

%load_ext tensorboard
%tensorboard --logdir {log_dir} 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Evaluate the model

In [ ]:
from tfx.components import ResolverNode
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing


model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 71
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [ ]:
# nb it always blesses on first run even if below threshold
import tensorflow_model_analysis as tfma

eval_config=tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(class_name='BinaryAccuracy'),
              tfma.MetricConfig(class_name='ExampleCount'),
              tfma.MetricConfig(class_name='AUC')
              ],
              thresholds={
                  'AUC':
                      tfma.config.MetricThreshold(
                          value_threshold=tfma.GenericValueThreshold(
                              lower_bound={'value': 0.65}),
                          change_threshold=tfma.GenericChangeThreshold(
                              direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                              absolute={'value': 0.01}))}
                          )])

In [ ]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 72
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: ../tfx/Evaluator/evaluation/72, id: 86)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: ../tfx/Evaluator/blessing/72, id: 87)]
        ))

In [ ]:
#NB TFMA visualizations will not run in Jupyter Lab
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

In [ ]:
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [ ]:
from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "../interactive-pipeline/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 73
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: ../tfx/Pusher/pushed_model/73, id: 88)]
        ))

### Extra stuff

In [ ]:
!mkdir -p ../tfx-9Apr/serving_model_dir

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# THEN REFRESH BROWSER PAGE!

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

In [ ]:
# Show data sliced by product
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

In [ ]:
# fairness indicators direct from pipeline
# https://colab.research.google.com/github/tensorflow/fairness-indicators/blob/master/fairness_indicators/examples/Fairness_Indicators_Lineage_Case_Study.ipynb